In [3]:
import boto3

import os

import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
jfolder = '../data/seattle-json/'

files = os.listdir(jfolder)
files = set(files)

In [6]:
### only run this code if you need to update from s3 bucket

files = set(os.listdir(jfolder))

# import sys
# reload(sys)  # Reload does the trick!
# sys.setdefaultencoding('UTF8')



# bucket_name = 'wanderwell-world'
# fnames = []

# s3 =  boto3.resource('s3')
# s3bucket = s3.Bucket(bucket_name)
# for name in s3bucket.objects.all():
#     fnames.append(name.key)

# len(fnames)

# for name in s3bucket.objects.all():
#     fname = name.key
#     if fname not in files and '.json' in fname :
#         print fname
#         s3.meta.client.download_file('wanderwell-world',
#                                      fname,
#                                     '{}{}'.format(jfolder,fname))
    

### FLATTENING THE DATAFRAME

In [7]:
len(os.listdir(jfolder))

6403

In [8]:
daylist = ['mon', 'tues', 'wed', 'thurs', 'fri', 'sat', 'sun']

poi_list = []
cat_set = set()
for bus in os.listdir(jfolder):
    if bus.split('.')[-1] == 'json':
            with open(jfolder+bus) as busfile:
                bus_json = json.load(busfile)
                cats = bus_json.pop('categories')
                ### seperate hours into daily columns
                if 'hours' in bus_json:
                    hours_dict = bus_json.pop('hours')[0]['open']
                else:
                    hours_dict = None
                for idx, day in enumerate(daylist) :
                    try : 
                        bus_json['hours_open_{}'.format(day)] = hours_dict[idx]['start']
                    except Exception: 
                        bus_json['hours_open_{}'.format(day)] = False
                    try : 
                        bus_json['hours_closed_{}'.format(day)] = hours_dict[idx]['end']
                    except Exception: 
                        bus_json['hours_closed_{}'.format(day)] = False
                ### seprate catagories into individual columns (3 total)
                for idx, val in enumerate(cats):
                    bus_json['category-{}'.format(idx)] = val['alias']
                    cat_set.add('cat: {}'.format(val['alias']))
                poi_list.append(pd.io.json.json_normalize(bus_json))
            continue

seattle_df = pd.concat(poi_list)

for cat in ['category-0','category-1','category-2'] :
    seattle_df[cat].fillna(False,inplace=True)

In [11]:
seattle_df_ = seattle_df.set_index('id')
seattle_df_['review_count'].nlargest(10)

id
pike-place-chowder-seattle                4440
paseo-caribbean-food-fremont-seattle-2    4305
piroshky-piroshky-seattle                 4033
serious-pie-seattle                       3195
pike-place-market-seattle                 2978
toulouse-petit-seattle                    2827
lola-seattle                              2787
umi-sake-house-seattle                    2661
the-pink-door-seattle-4                   2561
local-360-seattle-2                       1906
Name: review_count, dtype: int64

In [12]:
n_photos = seattle_df_['photos'].apply(lambda x : len(x))
n_photos.nlargest(30)
# n_photos.sum()

id
pike-place-market-seattle                    6347
space-needle-seattle                         3870
piroshky-piroshky-seattle                    3744
pike-place-chowder-seattle                   3425
toulouse-petit-seattle                       3055
din-tai-fung-seattle                         2650
the-pink-door-seattle-4                      2454
skycity-at-the-needle-seattle-3              2252
umi-sake-house-seattle                       2062
starbucks-seattle-88                         2018
beechers-handmade-cheese-seattle             1938
safeco-field-seattle                         1925
biscuit-bitch-seattle                        1873
the-walrus-and-the-carpenter-seattle         1698
the-crumpet-shop-seattle                     1691
serious-pie-seattle                          1680
pike-place-market-gum-wall-seattle           1654
le-panier-french-bakery-seattle              1584
local-360-seattle-2                          1561
paseo-caribbean-food-fremont-seattle-2       15

In [15]:
seattle_df['review_count'].sum()

636848

In [16]:
seattle_df['review_count'].mean()

99.49195438212779

In [17]:
seattle_df['category-0'].value_counts().nlargest(30)

coffee              381
pizza               142
newamerican         131
mexican             118
grocery             114
bakeries            113
vietnamese          106
hair                105
bars                102
sandwiches           96
thai                 87
parks                85
foodtrucks           85
autorepair           81
pubs                 73
generaldentistry     73
japanese             71
delis                66
italian              65
tradamerican         64
burgers              62
hotdogs              62
cafes                60
seafood              59
chinese              56
sushi                51
othersalons          50
realestateagents     50
lounges              47
convenience          47
Name: category-0, dtype: int64

### JSON FILE OF LIST OF CATAGORIES IS  defined below

In [19]:
catfile = '../data/categories.json'

with open(catfile) as cats:
    cat_son = json.load(cats)

cat_df = pd.DataFrame(cat_son)

cat_key = cat_df.set_index('alias')['parents']
cat_key_m = cat_key.apply(lambda x: pd.Series(x))
cat_key_m = cat_key_m[0]
cat_key_m.head()

alias
3dprinting      localservices
abruzzese             italian
absinthebars             bars
acaibowls                food
accessories           fashion
Name: 0, dtype: object

In [20]:
cat_df.head()

,alias,country_blacklist,country_whitelist,parents,title
0,3dprinting,NaN,NaN,[localservices],3D Printing
1,abruzzese,NaN,[IT],[italian],Abruzzese
2,absinthebars,NaN,[CZ],[bars],Absinthe Bars
3,acaibowls,"[IT, MX, PL, AR, TR, CL]",NaN,[food],Acai Bowls
4,accessories,NaN,NaN,[fashion],Accessories


#### EXTRACT ALL THROW AWAY CATAGORIES

In [21]:
cat_df.head()
# cat_df.columns
cat_key_m.value_counts().shape

(104,)

In [22]:
cats_df = cat_key_m.reset_index()
cats_df.columns = ['catagory', 'parent']
cats_df.dropna(inplace=True)

has_service = cats_df['parent'].str.contains('service')
has_shopping = cats_df['parent'].str.contains('shopping')
has_health = cats_df['parent'].str.contains('health')
has_edu = cats_df['parent'].str.contains('education')
has_fit = cats_df['parent'].str.contains('fitness')
has_auto = cats_df['parent'].str.contains('auto')

dumps = has_service + has_health  + has_auto + has_edu + has_fit

dump_cats = cats_df[dumps.values]
has_dump = set(dump_cats['catagory'].values)

print dump_cats.shape
dump_cats.head()

(401, 2)


/Users/travis/anaconda2/lib/python2.7/site-packages/pandas/computation/expressions.py:181: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  unsupported[op_str]))


,catagory,parent
0,3dprinting,localservices
8,acupuncture,health
10,adoptionservices,localservices
12,adultedu,education
15,aerialfitness,fitness


#### EXTRACT ALL FOOD CATAGORIES

In [23]:
cats_df = cat_key_m.reset_index()
cats_df.columns = ['catagory', 'parent']
food_bool = cats_df['parent'] == 'food'
rest_bool = cats_df['parent'] == 'restaurants'
all_food_bool = food_bool + rest_bool

food_cats = cats_df[all_food_bool]
has_food = set(food_cats['catagory'].values)

print food_cats.shape
food_cats.head()

(255, 2)


,catagory,parent
3,acaibowls,food
17,afghani,restaurants
18,african,restaurants
37,andalusian,restaurants
49,arabian,restaurants


#### EXTRACT ALL NIGHTLIFE CATAGORIES

In [24]:
cats_df = cat_key_m.reset_index()
cats_df.columns = ['catagory', 'parent']
club_bool = cats_df['parent'] == 'nightlife'
bar_bool = cats_df['parent'] == 'bars'
beer_bool = cats_df['parent'].str.contains('beer').fillna(False)
wine_bool = cats_df['parent'].str.contains('wine').fillna(False)
cocktail_bool = cats_df['parent'].str.contains('cocktail').fillna(False)


all_drink_bool = club_bool + bar_bool + beer_bool + wine_bool + cocktail_bool

nightlife_cats = cats_df[all_drink_bool]
has_nightlife = set(nightlife_cats['catagory'].values)

print nightlife_cats.shape
nightlife_cats.head()

(40, 2)


,catagory,parent
2,absinthebars,bars
13,adultentertainment,nightlife
26,airportlounges,bars
108,barcrawl,nightlife
110,bars,nightlife


In [25]:
len(has_nightlife)

40

#### EXTRACT ALL COFFEE CATAGORIES

In [26]:
cats_df = cat_key_m.reset_index()
cats_df.columns = ['catagory', 'parent']

has_coffee = cats_df['catagory'].str.contains('coffee').fillna(False)

coffee_cats = cats_df[has_coffee]
has_coffee = set(coffee_cats['catagory'].values)

print coffee_cats.shape
coffee_cats[coffee_cats['catagory'].str.contains('tea')].head()

(4, 2)


,catagory,parent
286,coffeeteasupplies,food


## ADD CATAGORIES TO DATAFRAME

In [27]:
seattle_df_ = seattle_df.copy()

seattle_df_['coffee'] = seattle_df_['category-0'].isin(has_coffee)
seattle_df_['coffee'] = seattle_df_['coffee'] + seattle_df_['category-1'].isin(has_coffee)
seattle_df_['coffee'] = seattle_df_['coffee'] + seattle_df_['category-2'].isin(has_coffee)


seattle_df_['food'] = seattle_df_['category-0'].isin(has_food)
seattle_df_['food'] = seattle_df_['food'] + seattle_df_['category-1'].isin(has_food)
seattle_df_['food'] = seattle_df_['food'] + seattle_df_['category-2'].isin(has_food)

seattle_df_['nightlife'] = seattle_df_['category-0'].isin(has_nightlife)
seattle_df_['nightlife'] = seattle_df_['nightlife'] + seattle_df_['category-1'].isin(has_food)
seattle_df_['nightlife'] = seattle_df_['nightlife'] + seattle_df_['category-2'].isin(has_food)

seattle_df_['dump'] = seattle_df_['category-0'].isin(has_dump)
seattle_df_['dump'] = seattle_df_['nightlife'] + seattle_df_['category-1'].isin(has_dump)
seattle_df_['dump'] = seattle_df_['nightlife'] + seattle_df_['category-2'].isin(has_dump)
seattle_df_['dump'] = seattle_df_['dump'] * np.invert(seattle_df_['food'])
seattle_df_['dump'] = seattle_df_['dump'] * np.invert(seattle_df_['coffee'])
seattle_df_['dump'] = seattle_df_['dump'] * np.invert(seattle_df_['nightlife'])

/Users/travis/anaconda2/lib/python2.7/site-packages/pandas/computation/expressions.py:181: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))


In [28]:
seattle_df_ = seattle_df_[seattle_df_['dump'] ==  False]

In [29]:
seattle_df_.shape

(6240, 47)

In [30]:
seattle_df_.dump.sum()

0

In [31]:
seattle_df_.food.sum()

3383

In [32]:
seattle_df_.coffee.sum()

485

In [33]:
seattle_df_.nightlife.sum()

2400

In [34]:
cat_key_m['florists']

u'flowers'

### Plot datapoints on google map file

In [46]:
from gmplot import GoogleMapPlotter

s_lat = '47.629458' 
s_long = '-122.336338'

lats = seattle_df['coordinates.latitude'].values
longs = seattle_df['coordinates.longitude'].values

gmap = GoogleMapPlotter(s_lat, s_long, 13)


gmap.plot?
# gmap.plot(lats, longs, 'cornflowerblue', edge_width=10)
# gmap.scatter(lats, longs, '#3B0B39', size=4, marker=False)
# gmap.scatter(lats, longs, 'k', marker=True)
# gmap.heatmap(lats, longs)

# gmap.draw("../data/mymap_seattle.html")
gmap.draw('../data/seattle.html')

In [ ]:
pwd

In [ ]:
for reviews, name in (seattle_df[['reviews', 'name']].values[:2,:]):
    print name

In [39]:
seattle_df_.columns

Index([              u'category-0',               u'category-1',
                     u'category-2',               u'category-3',
           u'coordinates.latitude',    u'coordinates.longitude',
                  u'display_phone',         u'hours_closed_fri',
               u'hours_closed_mon',         u'hours_closed_sat',
               u'hours_closed_sun',       u'hours_closed_thurs',
              u'hours_closed_tues',         u'hours_closed_wed',
                 u'hours_open_fri',           u'hours_open_mon',
                 u'hours_open_sat',           u'hours_open_sun',
               u'hours_open_thurs',          u'hours_open_tues',
                 u'hours_open_wed',                       u'id',
                      u'image_url',               u'is_claimed',
                      u'is_closed',        u'location.address1',
              u'location.address2',        u'location.address3',
                  u'location.city',         u'location.country',
         u'location.cross

In [48]:
coffee_df = seattle_df[seattle_df['category-0'].isin(has_coffee)]
coffee_df.columns

Index([              u'category-0',               u'category-1',
                     u'category-2',               u'category-3',
           u'coordinates.latitude',    u'coordinates.longitude',
                  u'display_phone',         u'hours_closed_fri',
               u'hours_closed_mon',         u'hours_closed_sat',
               u'hours_closed_sun',       u'hours_closed_thurs',
              u'hours_closed_tues',         u'hours_closed_wed',
                 u'hours_open_fri',           u'hours_open_mon',
                 u'hours_open_sat',           u'hours_open_sun',
               u'hours_open_thurs',          u'hours_open_tues',
                 u'hours_open_wed',                       u'id',
                      u'image_url',               u'is_claimed',
                      u'is_closed',        u'location.address1',
              u'location.address2',        u'location.address3',
                  u'location.city',         u'location.country',
         u'location.cross

In [49]:
from sklearn.cluster import KMeans

In [50]:
map_location = coffee_df[['coordinates.longitude', 'coordinates.latitude']]
# map_location = map_location[map_location['coordinates.longitude'] > 47.515]
# map_location = map_location[map_location['coordinates.longitude'] < 47.69468]
# map_location = map_location[map_location['coordinates.latitude'] < -122.420]
# map_location = map_location[map_location['coordinates.latitude'] > -122.222]



coffee_cluster = KMeans(n_clusters=9)

coffee_cluster.fit(map_location)

what_is_this = coffee_cluster.fit_predict(coffee_df[['coordinates.longitude', 'coordinates.latitude']])
clustered_lats = []
clustered_longs = []
for n, i in enumerate(what_is_this):
    lats = coffee_df[what_is_this == n]['coordinates.latitude'].values
    longs = coffee_df[what_is_this == n]['coordinates.longitude'].values
    clustered_lats.append(lats)
    clustered_longs.append(longs)

colors_list = ['crimson', 'darkseagreen', 
               'firebrick', 'dodgerblue', 
               'burlywood' 'gold',
               'indigo',  'navy', 
               'indianred', 'lawngreen']

longs = coffee_df['coordinates.longitude'].values
lats = coffee_df['coordinates.latitude'].values

s_lat = '47.629458' 
s_long = '-122.336338'

cent_long = coffee_cluster.cluster_centers_[:,0]
cent_lat = coffee_cluster.cluster_centers_[:,1]

gmap = GoogleMapPlotter(s_lat, s_long, 11.8)

for color, lats, longs in zip(colors_list, clustered_lats, clustered_longs):
    gmap.scatter(lats, longs, color, size=50, marker=False)

# gmap.plot(lats, longs, 'cornflowerblue', edge_width=10)
# gmap.scatter(lats, longs, '#3B0B39', size=40, marker=False)
# gmap.scatter(lats, longs, 'k', marker=True)
# gmap.heatmap(lats, longs)
c_longs = []
c_lats = []
for line in coffee_cluster.cluster_centers_:
    c_longs.append(line[0])
    c_lats.append(line[1])
gmap.scatter(c_lats, c_longs, 'hotpink', size=40, marker=True)

gmap.draw("../images/coffee_centers.html")